<a href="https://colab.research.google.com/github/ben-oxley/colab-opencl-experiments/blob/main/double_number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
!pip install pycuda
%load_ext nvcc_plugin


Thu May  4 11:55:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy
num_elements = 10000000
numbers = numpy.random.uniform(0,1000,num_elements)
numbers = numbers.astype(numpy.float32)


In [35]:
mod = SourceModule("""
  __global__ void doubleit(float *numbers)
  {
    for (int i = 0; i < 10000000/1024; i++){
      int idx = (int)threadIdx.x+i*1024;
      numbers[idx] *= 2;
    }
    
  }
  
  """)

In [40]:
numbers_doubled = [num*2 for num in numbers]
print(numbers[-2000:-1999])
print(numbers_doubled[-2000:-1999])

[932.2594]
[1864.518798828125]


In [41]:

numbers_gpu_buffer = cuda.mem_alloc(numbers.nbytes)
cuda.memcpy_htod(numbers_gpu_buffer, numbers)
func = mod.get_function("doubleit")
func(numbers_gpu_buffer, block=(1024,1,1))

numbers_doubled = numbers.astype(numpy.float32)
cuda.memcpy_dtoh(numbers_doubled,numbers_gpu_buffer)

print(numbers_doubled[-2000:-1999])

[1864.5188]
